In [2]:
import json

In [6]:
with open("../../data/dataset/source3_reentrancy.json", "r",  encoding="utf-8") as file:
    data = json.load(file)

In [7]:
data[0].keys()

dict_keys(['instruction', 'input', 'output'])

In [8]:
for record in data:
    contract = record['input']
    lines = contract.split("\n")
    numbered_lines = [f"{i+1}: {line}" for i, line in enumerate(lines)]
    record['input'] = "\n".join(numbered_lines)


In [10]:
print(record['input'])

1: contract ExampleFlashSwap is IUniswapV2Callee {
2:     IUniswapV1Factory immutable factoryV1;
3:     address immutable factory;
4:     IWETH immutable WETH;
5: 
6:     constructor(address _factory, address _factoryV1, address router) public {
7:         factoryV1 = IUniswapV1Factory(_factoryV1);
8:         factory = _factory;
9:         WETH = IWETH(IUniswapV2Router01(router).WETH());
10:     }
11: 
12:     // needs to accept ETH from any V1 exchange and WETH. ideally this could be enforced, as in the router,
13:     // but it's not possible because it requires a call to the v1 factory, which takes too much gas
14:     receive() external payable {}
15: 
16:     // gets tokens/WETH via a V2 flash swap, swaps for the ETH/tokens on V1, repays V2, and keeps the rest!
17:     function uniswapV2Call(address sender, uint amount0, uint amount1, bytes calldata data) external override {
18:         address[] memory path = new address[](2);
19:         uint amountToken;
20:         uint amount